**INPUT**: 
- topUSlakes.csv : ice phenology summary for 100 US lakes
    - columns include:
        - lakecode: unique identifier for lake
        - lake: lake name
        - start_date: first year of ice record
        - end_date: last year of ice record
        - lat: latitude 
        - lon: longitude
        - Elevation: elevation (m)


- topUSlakes_weather_stations: filtered list of NOAA weather stations within 50 km of lakes
    - the complete list of stations, f'mshr_enhanced_202102.txt' is available online from https://www.ncei.noaa.gov/access/homr/reports/mshr
- weather station data: a separate csv file for each weather station
    - saved in WEATHER_DIR
    - filename should be [GHCND_ID].csv (e.g., US1MNBW0013.csv)
    - these data can be downloaded from https://www.ncei.noaa.gov/access/services/data/v1?dataset=daily-summaries
        - API documentation: https://www.ncei.noaa.gov/support/access-data-service-api-user-documentation

**PARAMETERS**:
- delta_elevation = 100 (maximum allowed elevation difference between lake and weather station)

**OUTPUT**: 
- top100_extra_weather_all_deltaElev_100m.csv

    - columns include:
        - DATE
        - lakecode
        - lake
        - STATION
        - TMINMAX
        - SNOW
        - PRCP
        - SNWD
        - TSUN
        
        
**DEPENDENCIES**:
- ``kpmb_weather.py`` is found in `modules` directory (requirements: geopy, pandas, requests)
- pandas, matplotlib, IPython

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

import os
from IPython.display import clear_output

from kpmb_weather import find_stations, get_id


Define ICEMODELS_DIR, the directory where the ice phenology data are saved.

Define WEATHER_DIR, the directory where the downloaded weather csv files are saved.

In [ ]:
volume = 'My Passport for Mac'

ICEMODELS_DIR = Path(f'/Volumes/{volume}/IceModels/')
WEATHER_DIR = Path(f'/Volumes/{volume}/WeatherData/NOAA/csv/')

Set parameters for weather station search

In [ ]:
delta_elev_m = 100 # maximum allowed difference between lake and weather station elevation

delta_elev_ft = delta_elev_m*3.28084 # 100 metres converted to feet


Read in csv containing ice phenology data for lakes of interest

In [ ]:
nlakes = 100

dftopN = pd.read_csv(ICEMODELS_DIR/'topUSlakes.csv')


Read in list of all stations within 50 km of lake coordinate. The file ``topUSlakes_weather_stations.csv`` was created from ``mshr_enhanced_202102.txt`` (https://www.ncei.noaa.gov/access/homr/reports/mshr)

Important columns include:
- BEGIN_DATE and END_DATE
- NCDCSTN_ID
- latitude and longitude of station



In [ ]:
dfstations = pd.read_csv(ICEMODELS_DIR/'topUSlakes_weather_stations.csv')

Look for and remove stations that do not contain any temperature information

In [ ]:
for i,c in enumerate(dfstations.GHCND_ID.unique()):
    clear_output(wait=True)
    print(f"{i+1:4d}/{len(dfstations.GHCND_ID.unique())}")
    try:
        df_ = pd.read_csv(WEATHER_DIR/'{c}.csv')
        # look for stations that are missing temperature data
        if len(df_.dropna(how='all',axis=1).columns.intersection(['TMIN','TMAX','TAVG']))==0:
            dfstations.loc[dfstations.GHCND_ID==c,'temperature'] = False
        else:
            dfstations.loc[dfstations.GHCND_ID==c,'temperature'] = True
    except:
        continue

Select list of stations that have temperature information

In [ ]:
dfstations_temp = dfstations[dfstations.temperature==True]
#dfstations_temp

## Build meteorological time series

Build weather time series based on availability of TMINMAX (i.e., TMIN, TMAX and TAVG data).
- **NEW FEB 10 2022** All data in a given row is from the same weather station.

In [ ]:
dfworking = pd.DataFrame()
#dfworking_station = pd.DataFrame()

# run through list of lakes
for i,row in dftopN.iterrows():

    clear_output(wait=True)
    print(f"{i+1:2d}/100 - {row.lakecode}")
    lat,lon = row.lat, row.lon

    # elevation of lake
    elevation = row.Elevation *3.28084 # convert to feet since stations are in feet

    # row.start_date and row.end_date are the years for which we have ice phenology records
    #   need weather data for this same period
    #
    start_date = f'1 January {row.start_date}'
    end_date = f'31 December {row.end_date}'

    # finds all weather stations within 50 km of lat, lon
    #    with data between start_date and end_date,
    #    with a maximum delta elevation of 100 m.
    stations,_,distances = find_stations(lat,lon,dfstations_temp,
                                         verbose=False,
                      dist=50,elev=elevation, d_elev=delta_elev_ft,
                      start_date=start_date,end_date=end_date)
    
    # stations is a dictionary of {station_number: [list of timestamps]}
    # where station_number is NCDCSTN_ID
    #   - order is from station with most coverage to station with the least coverage
    #         within the date range specified
    
    # get list of station ID from closest to furthest
    other_stations = get_id(lat,lon, dfstations_temp, dist=50,elev=elevation,
                            d_elev=delta_elev_ft,getid='GHCND_ID')

    # start to build up time series, starting with an empty data frame
    #   which covers the entire range of the ice phenology record from 
    #   start_date to end_date
    df_lakecode = pd.DataFrame(index=pd.date_range(start_date,end_date))
    
    # CAVEAT: cycles from most to least coverage (but no specific mention of TMIN or TMAX) 
    #     so could be most SNOW coverage perhaps
    for key,value in stations.items():
        # key is station number NCDCSTN_ID
        # value is list of dates
        # order of stations is from most to least coverage
        
        #d1min = min(d1).strftime('%Y-%m-%d')
        #d1max = max(d1).strftime('%Y-%m-%d')
        # get list of GHCND_ID that match NCDCSTN_ID
        ghcnd_id = dfstations_temp[dfstations_temp.NCDCSTN_ID==key].GHCND_ID.dropna().unique()
        
        # cycle through all stations associated with this NCDCSTN_ID (key)
        for ghcnd in ghcnd_id:
            # read in data from file (all data gets read in)
            filename = WEATHER_DIR/f'{ghcnd}.csv'
            try:
                dfoo = pd.read_csv(filename)
            except:
                print('file not found error.')
                continue
            dfoo = dfoo.set_index('DATE')
            dfoo.index = pd.to_datetime(dfoo.index)
            
            # check quality attributes and remove bad data
            for var in ['TMIN','TMAX','TAVG']:
                dfoo[var] = dfoo[var].astype(float)
                ind_flag = ~dfoo[f'{var}_ATTRIBUTES'].astype(str).str.split(',').str[1].astype(str).str.strip().isin(['','nan'])
                dfoo.loc[ind_flag,var] = np.nan
            
            # calculate average temperature from TMIN and TMAX
            dfoo['TMINMAX'] = (dfoo['TMIN'] + dfoo['TMAX'])/2.
            
            # if TMIN and TMAX do not exist (i.e., they are NaN) replace with TAVG value
            ind = dfoo.TMINMAX.isnull() & ~dfoo.TAVG.isnull()
            dfoo.loc[ind,'TMINMAX'] = dfoo.loc[ind,'TAVG'].copy()
            
            station = dfoo.STATION.drop_duplicates().tolist()[0]
            
            # fills in blanks with data from other stations
            
            if df_lakecode.shape[1]==0:
                df_lakecode = df_lakecode.merge(dfoo,left_index=True,right_index=True, how='left')
                #df_station_record = df_lakecode.astype(str).copy()
                #df_station_record[df_station_record!='nan'] = station
            
                df_lakecode['TMINMAX'] = (df_lakecode['TMIN']+ df_lakecode['TMAX'])/2.
                ind = df_lakecode.TMINMAX.isnull() & ~df_lakecode.TAVG.isnull()
                df_lakecode.loc[ind,'TMINMAX'] = df_lakecode.loc[ind,'TAVG'].copy()
            
            else:
                # [old version]
                # df_lakecode= df_lakecode.fillna(dfoo)
                
                # fill in where TMINMAX is null
                ind_rows = df_lakecode.TMINMAX.isnull() & ~dfoo.TMINMAX.isnull()
            
                df_lakecode.loc[ind_rows,:] = dfoo.loc[ind_rows,:].copy()
                
                #ind_new = df_lakecode.isnull() & ~dfoo.isnull() #& (df_station_record=='nan')
                #df_station_record[ind_new] = station
                #
                
    # now fillna using other_stations    
    for ghcnd in other_stations:
        #print(ghcnd)
        filename = WEATHER_DIR/f'{ghcnd}.csv'
        try:
            dfoo = pd.read_csv(filename,low_memory=False)
        
        except:
            print(ghcnd)
            print('file not found error.')
            continue
        if len(dfoo)==0:
            continue
            
        #print(ghcnd, dfoo.DATE.min(),dfoo.DATE.max())
        dfoo = dfoo.set_index('DATE')
        dfoo.index = pd.to_datetime(dfoo.index)
        
        # check quality attributes and remove bad data
        for var in ['TMIN','TMAX','TAVG']:
            dfoo[var] = dfoo[var].astype(float)
            ind_flag = ~dfoo[f'{var}_ATTRIBUTES'].astype(str).str.split(',').str[1].astype(str).str.strip().isin(['','nan'])
            dfoo.loc[ind_flag,var] = np.nan

        dfoo['TMINMAX'] = (dfoo['TMIN'] + dfoo['TMAX'])/2.
        ind = dfoo.TMINMAX.isnull() & ~dfoo.TAVG.isnull()
        dfoo.loc[ind,'TMINMAX'] = dfoo.loc[ind,'TAVG'].copy()
        
        #station = dfoo.STATION.drop_duplicates().tolist()[0]
        if df_lakecode.shape[1]==0:
            df_lakecode = df_lakecode.merge(dfoo,left_index=True,right_index=True, how='left')
            
            df_lakecode['TMINMAX'] = (df_lakecode['TMIN']+ df_lakecode['TMAX'])/2.
            ind = df_lakecode.TMINMAX.isnull() & ~df_lakecode.TAVG.isnull()
            df_lakecode.loc[ind,'TMINMAX'] = df_lakecode.loc[ind,'TAVG'].copy()
            
            #df_station_record = df_lakecode.astype(str).copy()
            #df_station_record[df_station_record!='nan'] = station
        else:
            # [old version] filling blanks with data from different stations; 
            # so SNOW and TMINMAX for example may be from
            #     different stations... (record station!)
            # df_lakecode= df_lakecode.fillna(dfoo)
            
            # consistently fill in data only if TMINMAX is missing
            ind_rows = df_lakecode.TMINMAX.isnull() & ~dfoo.TMINMAX.isnull()
            
            df_lakecode.loc[ind_rows,:] = dfoo.loc[ind_rows,:].copy()
            
    #df_station_record['lakecode']= row.lakecode
    #df_station_record['lake'] = row.lake
    #df_station_record = df_station_record.reset_index().rename({'index':'DATE'},axis=1)
    #dfworking_station = dfworking_station.append(df_station_record,ignore_index=True)
    
    df_lakecode['lakecode']= row.lakecode
    df_lakecode['lake'] = row.lake
    df_lakecode = df_lakecode.reset_index().rename({'index':'DATE'},axis=1)
    
    dfworking = pd.concat([dfworking, df_lakecode],ignore_index=True)
    

### Summary of lakes and stations in meteorological time series
Look at all time series, recording the occurrence of use (in days) for each of the weather stations.

In [ ]:
dfworking.groupby(['lakecode','STATION']).DATE.count()

In [ ]:
#dfworking.set_index('DATE').SNOW.plot()
dfworking.STATION.unique().shape

### QA/QC of meteorological time series

List of variables:

- (AWDR : wind direction)
- AWND (m/s) : average daily wind speed
- PRCP (mm): precipitation
- SNOW (mm): snow
- SNWD (mm): snow depth
- TAVG (C): average temperature?
- THIC (mm): thickness of ice on water
- TMAX (C): maximum temperature
- TMIN (C): minimum temperature
- TOBS (C): temperature at the time of observation
- TSUN (minutes): daily total sunshine

#### 1. Check attribute columns (Meas,Qual,Source,Time) and remove data with a non-blank Qual flag 

In [ ]:
# copy list of all stations and daily meteorological data to dfresult
dfresult = dfworking.copy()

for c in dfworking.columns:
    if 'ATTRIBUTES' in c:
        if len(dfworking[c].dropna())>0:
            new_c = f"{c}_QUALITY"
            print(new_c)
            
            # Add new columns containing data quality flags only
            dfresult[new_c] = dfworking[c].dropna().str.split(',').str[1]
            dfresult[new_c] = dfresult[new_c].str.strip().replace('',np.nan)
            
            # Remove flagged data from table
            ind = ~dfresult[new_c].isnull()
            dfresult.loc[ind,c.replace('_ATTRIBUTES','')] = np.nan
            

#### 2. Convert columns to float dtype, date and sort by lakecode and date

In [ ]:
for c in dfresult.columns:
    try:
        dfresult[c] = dfresult[c].astype(float)
    except:
        print(c)

dfresult.DATE = pd.to_datetime(dfresult.DATE)

dfresult = dfresult.sort_values(['lakecode','DATE'])

#### 3. Re-calculate daily average temperatures


$$T_{\rm avg} = \frac{T_{\rm min}+T_{\rm max}}{2}$$


- fill in missing (or low quality) data with TAVG, if the value exists

In [ ]:
dfresult['TMINMAX'] = (dfresult.TMIN + dfresult.TMAX)/2.

# confirm that average is null where it should be
ind = dfresult.TMIN.isnull() | dfresult.TMAX.isnull() 

display(dfresult.loc[ind,'TMINMAX'].dropna())

dfresult.loc[ind,'TMINMAX'] = np.nan

# fill in blanks with TAVG if it exists

ind = dfresult.TMINMAX.isnull() & ~dfresult.TAVG.isnull()

dfresult.loc[ind,'TMINMAX'] = dfresult.loc[ind,'TAVG']


#### 5. Look at snow and precipitation

a) How are snow and precipitation related?
- it seems that they are independent
- PRCP is "rain, melted snow"




In [ ]:
dfresult.plot(x='PRCP',y='SNOW',ls='none',marker='.')

b) Check that it is cold enough for snow.
- looking at snowfall when average temperature is above 0 degrees C.
- when there is a warmer average daily temperature, there is less snow

In [ ]:
ind = (dfresult.SNOW>0) & (dfresult.TMINMAX>0)

x = dfresult[ind].SNOW
y = dfresult[ind].TMINMAX
plt.plot(x,y, ls='none',marker='.')
plt.ylabel('Temperature (\N{DEGREE SIGN}C)')
plt.xlabel('Amount of snow fall (mm)')

In [ ]:
#ind = (dfresult.TMINMAX>15) & (dfresult.SNOW > 0)
#dfresult[ind].dropna(how='all',axis=1)

In [ ]:
dfresult.describe()

### Interpolate missing/flagged data in meteorological time series

Fill in blanks. Linearly interpolate over blank days, up to 3 consecutive NaN.

- should be OK for temperature, snow depth
- precipitation and snow: may not make sense to do this.

In [ ]:
dfresult_filled = dfresult.copy()

# run through each lake's timeseries
for name,group in dfresult[['DATE','STATION','lakecode','lake','TMINMAX','SNOW','PRCP','SNWD','TSUN']].groupby('lakecode'):
    clear_output(wait=True)
    print(name)
    # cycle through TMINMAX, SNWD and TSUN, interpolating each
    for c in ['TMINMAX',
              #'SNOW','PRCP', # can't justify interpolating snow and precipitation
              'SNWD', # snow depth would make sense to interpolate between two days.
              'TSUN' # number of sun minutes can also be justified for interpolation
             ]:
        ts = group[c].astype(float)
        
        ts_filled = ts.interpolate(method='linear',limit=3, limit_area='inside')
        ind = (ts!=ts_filled) & (~ts_filled.isnull())
        print(c,ind.sum())
        # replace old time series with this new interpolated time series
        dfresult_filled.loc[group.index,c] = ts_filled

In [ ]:
#dfresult[dfresult.lakecode=='DMR2'].set_index('DATE').tail(8380).head(10).dropna(how='all',axis=1)

### Some lakes still have incomplete meteorological time series
Interpolation may not completely fill the meteorological record.

These lakes have incomplete daily temperature records in the period from 1960 to present. Incomplete years are listed after the lakecode and lake name.

All other lakes have complete temperature records in this 1960-present time window.

In [ ]:
for name,group in dfresult_filled[dfresult_filled.TMINMAX.isnull() & (dfresult_filled.DATE.dt.year>=1960)].groupby('lakecode'):
    print(name, group.lake.drop_duplicates().to_list()[0], group.DATE.dt.year.unique())

In [ ]:
# check completion of columns
#ind = dfresult_filled.TMINMAX.isnull() #& ~dfresult_filled.TOBS.isnull()
#display(dfresult_filled[ind])

### Save continuous time series
- changed to "extra" weather in filename

In [ ]:
dfresult_filled.to_csv(ICEMODELS_DIR/f'top{nlakes}_extra_weather_all_deltaElev_{delta_elev_m}m.csv',index=False)


___
### Extra QA/QC: Check weather station elevation vs lake elevation
All lakes should be within 100 m of the stations used to build their daily meteorological time series.

In [ ]:
dfresult = pd.DataFrame()

for stn,group in dfresult_filled.groupby('STATION'):
    dfmerge = group.copy()
    
    dfstn = dfstations.loc[dfstations.GHCND_ID==stn,['BEGIN_DATE','END_DATE','GHCND_ID','ELEV_GROUND','ELEV_GROUND_UNIT']]

    # backfill/forward fill if data is missing for given years, assuming elevation hasn't changed
    dfstn = dfstn.fillna(method='bfill').fillna(method='ffill')
    
    dfstn.BEGIN_DATE = dfstn.BEGIN_DATE.replace(10101,17000101)
    dfstn.END_DATE = dfstn.END_DATE.replace(99991231,21001231)
    
    newindex = pd.to_datetime(dfstn.set_index('BEGIN_DATE').index.tolist(),format='%Y%m%d')
    dfstn.index = newindex
    
    # create a continuous time series
    dfstn_new = dfstn[~dfstn.index.duplicated(keep='first')]
    dfstn_new = dfstn_new.resample('D').ffill()
    # find where elevation of station changes
    dfstn_new = dfstn_new[dfstn_new.ELEV_GROUND.diff() != 0]
    # redefine END_DATE based on when elevation changes (i.e., day before next BEGIN_DATE)
    dfstn_new['END_DATE'] = dfstn_new.index #-pd.to_timedelta('1 day')
    dfstn_new['END_DATE'] = dfstn_new['END_DATE'].shift(-1).dt.strftime('%Y%m%d')
    dfstn_new = dfstn_new.reset_index(drop=True)
    
    # make sure all dates before and after are included in case the station summary file is missing dates
    dfstn_new.iloc[0,0] = 10101
    dfstn_new.iloc[-1,1] = 99991231
    dfstn = dfstn_new.copy()
    
    if len(dfstn)==1:
        dfoo = group.reset_index().merge(dfstn,left_on='STATION',right_on='GHCND_ID').set_index('index')
    elif len(dfstn)==0:
        print('missing')
        break
    else:
        df_elev = pd.DataFrame()
        for i,row in dfstn.iterrows():
            if row.BEGIN_DATE==10101:
                #display(dfstn)
                begin_date = pd.to_datetime('1700-01-01')
            else:
                begin_date = pd.to_datetime(row.BEGIN_DATE, format='%Y%m%d')
            if row.END_DATE==99991231:
                end_date = pd.to_datetime('2100-12-31')
            else:
                # subtract one day so inclusive between statement below works as expected
                end_date = pd.to_datetime(row.END_DATE, format='%Y%m%d') - pd.to_timedelta('1 day')
            #print(begin_date,end_date)
            ind = group.DATE.between(begin_date, end_date, inclusive='both')
            solution_ =  group[ind].reset_index().merge(row.to_frame().T, left_on='STATION',right_on='GHCND_ID',how='left').set_index('index')
            #df_elev = df_elev.append(solution_[row.index].dropna())
            
            df_elev = pd.concat([df_elev, solution_[row.index].dropna()])
            
            #if len(df_elev) == len(group):
            #    break
                
        try:
            assert len(df_elev) == len(group)
        except AssertionError:
            print('Assertion Error.')
            display(group.loc[[j for j in group.index if j not in df_elev.index]])
            break

        dfoo = group.merge(df_elev, left_index=True, right_index=True,how='left')
    dfresult = pd.concat([dfresult, dfoo])
    #dfresult = dfresult.append(dfoo)

# finally append missing station rows
#dfresult = dfresult.append(dfresult_filled[dfresult_filled.STATION.isnull()])
dfresult = pd.concat([dfresult, dfresult_filled[dfresult_filled.STATION.isnull()]])



In [ ]:
dfresult2 = dfresult.merge(dftopN.loc[:,['lakecode',
                                              'Elevation']].drop_duplicates().rename({'Elevation':'Lake_elevation'},axis=1),
               left_on='lakecode',right_on='lakecode',validate='many_to_one',how='left')

dfresult2['Elevation_difference'] = dfresult2.ELEV_GROUND/3.28084 - dfresult2.Lake_elevation

dfresult2[np.abs(dfresult2.Elevation_difference) >100].drop_duplicates(['STATION','lakecode'])

In [ ]:
x,y = dfresult2.drop_duplicates(['lakecode','STATION'])[['Lake_elevation', 'ELEV_GROUND']].T.values

ydiff = y/3.28084 - x
fig,ax = plt.subplots()
ax.plot(x,ydiff,marker='.',ls='none')
ax.set_ylabel('Station-lake elevation difference (m)')
ax.set_xlabel('Lake elevation (m)')
#ax.plot([100,700],[100,700],color='k',lw=0.5)
ax.axhline(0,color='k',lw=0.5)